In [1]:
import pandas as pd
import os
import numpy as np

## Preliminary work

In [2]:
os.listdir("./")

['.git',
 '.ipynb_checkpoints',
 'BOWDescriptions.csv',
 'GradientBoostedTrees Popularity.ipynb',
 'LogisticRegression Popularity.ipynb',
 'Models(Will).ipynb',
 'Prediction using textual data.ipynb',
 'PreliminaryWork.ipynb',
 'Preprocessing Steam game description.ipynb',
 'preprocess_project.csv',
 'Project Preprocessing (1).ipynb',
 'Project Preprocessing.ipynb',
 'project_preprocess.csv',
 'RandomForest Popularity.ipynb',
 'Regression Models.ipynb',
 'steam.csv',
 'steam_description_data.csv',
 'TFIDFDescriptions.csv',
 'Untitled.ipynb']

In [3]:
df=pd.read_csv("steam.csv")

In [4]:
df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [5]:
df.dtypes

appid                 int64
name                 object
release_date         object
english               int64
developer            object
publisher            object
platforms            object
required_age          int64
categories           object
genres               object
steamspy_tags        object
achievements          int64
positive_ratings      int64
negative_ratings      int64
average_playtime      int64
median_playtime       int64
owners               object
price               float64
dtype: object

## Convert the owner range to numbers and then bin them.

In [6]:

df.owners=df.owners.astype("string")
owners=df.owners.str.split("-")

result=[]
for i in range(len(owners)):
    low=int(owners[i][0])
    high=int(owners[i][1])
    result.append((high+low)/2)

df["Owners"]=result

df.head()



,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,Owners
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,15000000.0
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99,7500000.0
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99,7500000.0
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99,7500000.0
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99,7500000.0


In [7]:
df.Owners.value_counts()

10000.0        18596
35000.0         3059
75000.0         1695
150000.0        1386
350000.0        1272
750000.0         513
1500000.0        288
3500000.0        193
7500000.0         46
15000000.0        21
35000000.0         3
75000000.0         2
150000000.0        1
Name: Owners, dtype: int64

In [8]:
#df["Popularity"]=pd.qcut(df['Owners'],q=3, labels=["Quite Popular", "Very Popular", "Cult Game"])
df["popularity"]=pd.cut(df["Owners"],bins=[0,20000,100000, 200000000], labels=["Low popularity","Average", "Popular"])

df["popularity"].value_counts()

Low popularity    18596
Average            4754
Popular            3725
Name: popularity, dtype: int64

In [20]:
df.shape

(27075, 18)

## Baseline error for popularity


In [11]:
df.popularity.value_counts()

Low popularity    18596
Average            4754
Popular            3725
Name: popularity, dtype: int64

In [12]:
1-df.popularity.value_counts()["Low popularity"]/df.shape[0]

0.31316712834718374

## Baseline error for positive ratings

In [13]:
y=df.positive_ratings

In [14]:
def MAE(y, yh):
#     for i in range(len(prediction)):
#         total+=abs(prediction[i]-prediction[i])
#     return total/len(prediction)
    return (np.abs(y-yh)).mean()


def MSE(y, yh):
#     total=0
#     for i in range(len(prediction)):
#         total+=(prediction[i]-prediction[i])**2
#     return total/len(prediction)   
    return ((y-yh)**2).mean()


In [15]:

yh=df.positive_ratings.mean()
MSE(y,yh)

360558338.2980273

In [16]:
yh=df.positive_ratings.median()
MAE(y, yh)

992.3237673130194

In [17]:
df=pd.read_csv("steam.csv")
y=df["average_playtime"]

In [18]:
yh=df["average_playtime"].mean()
MSE(y, yh)

3337945.079999253

In [19]:
yh=df["average_playtime"].median()
MAE(y, yh)

149.80494921514313